In [1]:
%run "U1. Imports.ipynb"
football_path = r"C:\Users\james\Documents\NFL\Data"

In [2]:
todaysdate

'20231012'

### All Contests

In [3]:
# Create dataframe with today's contest
def contests(date=todaysdate):
    # Contest URL
    url = "https://www.draftkings.com/lobby/getcontests?sport=NFL"

    # Extract JSON data
    response = requests.get(url)
    json_data = response.json()

    # Extract the "Contests" data
    contests_data = json_data["Contests"]

    # Create an empty list to store the extracted data
    rows = []

    # Iterate over each contest
    for contest in contests_data:
        contest_dict = {}

        # Extract the desired fields from the contest
        contest_dict["Name"] = contest["n"]
        contest_dict["Cash Prize"] = contest["pd"].get("Cash", None)
        contest_dict["Entry Fee"] = contest["a"]
        contest_dict["contestKey"] = contest["id"]
        contest_dict["draftGroupId"] = contest["dg"]
        contest_dict['contestDate'] = contest["sd"]
        contest_dict['contestDate'] = contest_dict['contestDate'].replace("/Date(", "").replace(")/","")
        contest_dict['contestTime'] = contest["sdstring"]
        
        # Append the extracted data to the list
        rows.append(contest_dict)

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(rows)
    df['contestDate'] = pd.to_numeric(df['contestDate'])
    
    df['contestDate'] = df['contestDate'] / 1000  # Convert milliseconds to seconds
    df['contestDate'] = df['contestDate'].apply(datetime.datetime.fromtimestamp)
    
    # Sort by date
    df.sort_values('contestDate', ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

### Selected Contests

In [4]:
# Choose contests of interest
def daily_contests(date, contest_name, exclusion):
    # Read in contests
    df = pd.read_csv(os.path.join(football_path, "Contests", "A. Daily Contests", f"Contests {date}.csv"))
    
    # Create an empty list to store the results
    results = []

    # Iterate over the rows of the dataframe
    for index, row in df.iterrows():
        name = row["Name"]
        # Keep "Four-Seamer" contests that aren't single-game contests
        if contest_name in name and exclusion not in name:
            contest_key = row["contestKey"]
            draft_group_id = row["draftGroupId"]
            contestTime = row['contestTime']
            results.append({"Name": name, "contestKey": contest_key, "draftGroupId": draft_group_id, "contestTime": contestTime})

    # Convert the results to a dataframe
    df = pd.DataFrame(results)

    return df

In [5]:
# Extract game info (used to determining away and home teams, date, time)
def get_game_info(row):
    # Define the time zone conversion
    source_timezone = pytz.timezone('UTC')
    target_timezone = pytz.timezone('America/New_York')
    
    # Extract competitions
    competitions = row.get('competitions', [])
    if competitions:
        name_display = competitions[0].get('nameDisplay', [])
        if len(name_display) >= 3:
            # Extract team names
            team1 = name_display[0]['value']
            team2 = name_display[2]['value']
            # And start time
            start_time = dateutil.parser.parse(competitions[0]['startTime']).astimezone(target_timezone).strftime('%m/%d/%Y %I:%M%p')
            # Convert to typical away@home datetime ET format
            return f"{team1}@{team2} {start_time} ET"
    return ""

In [6]:
# This creates DKSalaries
def draftables(draftGroupId):
    # Salary URL
    url = f'https://api.draftkings.com/draftgroups/v1/draftgroups/{draftGroupId}/draftables'
    
    # Extract JSON data
    response = requests.get(url)
    json_data = response.json()
    
    # Access the "draftables" key in the JSON data
    draftables = json_data["draftables"]

    # Convert the "draftables" data to a DataFrame
    df = pd.DataFrame(draftables)


    # print(df)
    
    # Extracting and formatting game information
    df["Game Info"] = df.apply(get_game_info, axis=1)
    df["AvgPointsPerGame"] = df["draftStatAttributes"].apply(lambda x: x[0]["value"])
    df['Name + ID'] = df['displayName'] + " (" + df['draftableId'].astype('str') + ")" 
    df["Roster Position"] = df["position"].apply(lambda x: "P" if x in ["SP", "RP"] else x)

    # Rename to match salary download files
    df.rename(columns={'position':'Position', 'displayName': 'Name', 'salary':'Salary', 'teamAbbreviation':'TeamAbbrev', 'draftableId':'ID'}, inplace=True)

    # Select relevant columns
    df = df[['Position', 'Name + ID', 'Name', 'ID', 'Roster Position', 'Salary', 'Game Info', 'TeamAbbrev', 'AvgPointsPerGame', 'playerId']]

    df.drop_duplicates(['Name', 'TeamAbbrev'], inplace=True)
    df.reset_index(inplace=True, drop=True)

    # Print the resulting DataFrame
    return df

In [7]:
# Not sure these are actually necessary anymore, although I wouldn't get contest size from leaderboard
def payouts(contestKey):
    # Payout URL
    url = f"https://api.draftkings.com/contests/v1/contests/{contestKey}?format=json"
    
    # Extract JSON data
    response = requests.get(url)
    json_data = response.json()
    
    # Extract minPosition, maxPosition, and payoutDescription
    payouts = json_data['contestDetail']['payoutSummary']
    data = []
    for payout in payouts:
        min_pos = payout['minPosition']
        max_pos = payout['maxPosition']
        payout_desc = payout['payoutDescriptions'][0]['payoutDescription']
        entry_fee = json_data['contestDetail']['entryFee']
        entries = json_data['contestDetail']['entries']
        max_entries_per_user = json_data['contestDetail']['maximumEntriesPerUser']
        draft_group_id = json_data['contestDetail']['draftGroupId']
        contest_key = json_data['contestDetail']['contestKey']
        contest_start_time = json_data['contestDetail']['contestStartTime']
        name = json_data['contestDetail']['name']
        data.append([min_pos, max_pos, payout_desc, entry_fee, entries, max_entries_per_user, draft_group_id, contest_key, contest_start_time, name])

    # Create a dataframe
    df = pd.DataFrame(data, columns=['minPosition', 'maxPosition', 'payoutDescription', 'entryFee', 'entries', 'maximumEntriesPerUser', 'draftGroupId', 'contestKey', 'contestStartTime', 'name'])

    return df

In [8]:
# This exports contest results and converts zip to csv and saves in proper location
# It creates two copies - one with contestKey (DraftKing's default) and another with the date
# This could be run for multiple days if you loop over contest history.
def results(contestKey):
    # Standings CSV link
    url = f"https://www.draftkings.com/contest/exportfullstandingscsv/{contestKey}"

    # Open in a new tab (same window)
    javascript_code = f"window.open('{url}', '_blank');"
    display(Javascript(javascript_code))

    # Specify the path to the Downloads directory
    downloads_folder = r'C:\Users\james\Downloads'

    # Get a list of all files in the Downloads directory
    files = os.listdir(downloads_folder)

    # Filter the list to include only zip files
    zip_files = [file for file in files if file.endswith('.zip')]

    # Sort the zip files by modification time (most recent first)
    sorted_zip_files = sorted(zip_files, key=lambda x: os.path.getmtime(os.path.join(downloads_folder, x)), reverse=True)

    # Look at zip files
    if sorted_zip_files:
        # Select the most recent zip file
        most_recent_zip_file = sorted_zip_files[0]

        # Specify the path to the most recent zip file
        zip_file_path = os.path.join(downloads_folder, most_recent_zip_file)

        # Specify the path to the destination folder where you want to save the copied file
        destination_folder = r'C:\Users\james\Documents\NFL\Data\Results\A. Contest Results'

        # Open the zip file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            # Get a list of all files in the zip file
            zip_files_list = zip_ref.namelist()

            # Extract the first file from the zip file to the destination folder
            file_to_copy = zip_files_list[0]
            zip_ref.extract(file_to_copy, destination_folder)

        print('File copied and renamed successfully!')
    else:
        print('No zip files found in the Downloads directory.')

In [9]:
# This scrapes the leaderboard
# This won't have every entrant, but it will have enough
# Note: These should be unnecessary when we have full results but could be useful otherwise
def leaderboard(contestKey):
    # Leaderboard URL
    url = f"https://api.draftkings.com/scores/v1/leaderboards/{contestKey}?format=json&embed=leaderboard"
    
    # Open in a new tab (same window)
    javascript_code = f"window.open('{url}', '_blank');"
    display(Javascript(javascript_code))

    # Wait for it to load
    time.sleep(4)

    # Simulate pressing Ctrl + S
    pyautogui.hotkey('ctrl', 's')

    # Wait to it to load
    time.sleep(2)

    # Simulate pressing Enter
    pyautogui.press('enter')
    
    # Wait for it to save
    time.sleep(1)
    
    # Close
    pyautogui.hotkey('ctrl', 'f4')

In [10]:
# This makes the leaderboard usable
def clean_leaderboard(contestKey):
    # Specify the path to your JSON file
    json_file_path = os.path.join(r"C:\Users\james\Documents\MLB\Data", "Y. One Time Use", f"{contestKey}.json")

    # Load the JSON data
    with open(json_file_path, 'r') as file:
        json_data = json.load(file)

    # Extract the required information from the JSON data
    leaderboard = json_data['leaderBoard']

    # Extract necessary information
    data = []
    for entry in leaderboard:
        row = {
            'rank': entry['rank'],
            'userName': entry['userName'],
            'fantasyPoints': entry['fantasyPoints'],
            'contestKey': entry['contestKey'],
            'draftGroupId': entry['draftGroupId'],
            'winningValue': entry.get('winnings', [{'value': None}])[0].get('value')
        }
        data.append(row)

    # Convert the extracted data to a DataFrame
    df = pd.DataFrame(data)
    df['winningValue'].fillna(0, inplace=True)
    
    return df

In [ ]:
def get_today(date=todaysdate):
    # Read in today's contests
    todays_contests = daily_contests(date, "Millionaire", "Sat")
    
    # Loop over contests to scrape payouts
    for contestKey in todays_contests['contestKey']:
        contest_payouts = payouts(contestKey)
        contest_payouts.to_csv(os.path.join(football_path, "Payouts", f"Payouts {contestKey}.csv"), index=False)
        
        time.sleep(5)

        
    # Loop over draft groups to scrape salaries
    for draftGroupId in todays_contests['draftGroupId'].unique():
        player_salaries = draftables(draftGroupId)
        player_salaries.to_csv(os.path.join(football_path, "Salaries",  f"Salaries {draftGroupId}.csv"), index=False, encoding='iso-8859-1')
        
        time.sleep(5)

In [ ]:
all_contests = contests(todaysdate)
all_contests.to_csv(os.path.join(football_path, "Contests", "A. Daily Contests", f"Contests {todaysdate}.csv"), index=False)

In [ ]:
selected_contests = daily_contests(todaysdate, "Millionaire", "Sat")
print(selected_contests.to_string())

In [ ]:
get_today(todaysdate)